# Training and Evaluation

We will take a first-pass at evaluating or technique to start understanding its efficacy. We will existing CNN architectures and evaluate its performance on our interested categories with and without using our interested categories.

In [1]:
import cv2
import datetime
from matplotlib import pyplot as plt
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_addons as tfa
print('TensorFlow Version: ', tf.__version__)

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

np.random.seed(123)

TensorFlow Version:  2.1.0


In [2]:
# Set hyperparameters for training & validation
INPUT_SHAPE = (64, 64, 3)
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.10
TRAIN_STEPS_PER_EPOCH = 5000
TEST_STEPS = 500
NUM_EPOCHS = 5
# Currently reduced
NUM_LABELS=22

In [3]:
# Define utilities and helper functions
# NOTE: Copied from clustering NB
def load_metadata(filename):
    with open(filename, 'r') as f:
        return [x.strip().split('\t') for x in f.readlines()]
    
@tf.function
def decode_img(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [64, 64])

@tf.function
def load_image_data(path, label):
    img_data = tf.io.read_file(path)
    img = decode_img(img_data)
    return img, label
    
def load_labels(metadata):
    labels = np.array([x[1] for x in metadata])
    distinct_labels = np.array([[x] for x in set(labels)])
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(distinct_labels)
    y_train = encoder.transform([[x] for x in labels])
    return (y_train, encoder)

# Load labels with an existing encoder
def load_labels_with_encoder(metadata, encoder):
    return encoder.transform([[x[1]] for x in metadata])

In [4]:
# Create functions for three models: (i) custom, simple CNN, (ii) MobileNet + FCs, and (iii) VGG16 + FCs
def get_simplecnn(input_shape=INPUT_SHAPE):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(512, (3, 3), (1, 1), input_shape=input_shape, activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(512, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_LABELS, activation='softmax'),
    ])

def get_mobilenet(input_shape=INPUT_SHAPE):
    application = tf.keras.applications.MobileNet(input_shape=input_shape, include_top=False)
    for i in range(len(application.layers)):
        application.layers[i].trainable = False
        
    return tf.keras.Sequential([
        application,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_LABELS, activation='softmax')
    ])

def get_vgg16(input_shape=INPUT_SHAPE):
    application = tf.keras.applications.VGG16(input_shape=input_shape, include_top=False)
    for i in range(len(application.layers)):
        application.layers[i].trainable = False
        
    return tf.keras.Sequential([
        application,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_LABELS, activation='softmax')
    ])

In [5]:
# simplecnn = get_simplecnn()
# simplecnn.summary()

In [6]:
mobilenet = get_mobilenet()
mobilenet.summary()

/home/dennis/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 2, 2, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

In [17]:
# vgg16 = get_vgg16()
# vgg16.summary()

In [5]:
# TODO: Remove hardcoding
print('Loading data into memory...')
train_metadata = load_metadata('./metadata_output/filtered_train_metadata.txt')
(y_train, encoder) = load_labels(train_metadata)

# Interested indices for test data filtering
interested_categories = ['n01882714', 'n04562935']
interested_one_hot = encoder.transform([[x] for x in interested_categories])
interested_indices = np.array([x[1] for x in np.argwhere(interested_one_hot == 1)])
print('Done.')

Loading data into memory...
Done.


In [6]:
# Encoding sanity checks;
# assert(len(train_metadata) == len(y_train))
# assert(len(set(y_train)) == 200)
assert(np.count_nonzero(y_train == 1) == len(train_metadata))
# print(y_train)

In [7]:
def join_paths_and_labels(train_metadata, y_train):
    return [(train_metadata[x][0], y_train[x]) for x in range(len(y_train))]

# TODO: This is sort of weird/dangerous.. we shouldn't be using a global for this
# since it changes per training run
num_validation = 0
def shuffle_and_split_data(train_metadata, y_train):
    global num_validation
    # Get all data
    paths_and_labels = join_paths_and_labels(train_metadata, y_train)
    print('Num. Total Images: ', len(paths_and_labels))

    # Split data into train and validation sets
    np.random.shuffle(paths_and_labels)
    num_validation = int(len(paths_and_labels) * VALIDATION_SPLIT)
    train_paths_and_labels = paths_and_labels[num_validation:]
    validation_paths_and_labels = paths_and_labels[:num_validation]
    print('Num. Train Images: ', len(train_paths_and_labels))
    print('Num. Validation Images: ', len(validation_paths_and_labels))
    
    return (train_paths_and_labels, validation_paths_and_labels)

## (1) BASELINE MODEL: VGG16

In [21]:
(train_paths_and_labels, validation_paths_and_labels) = shuffle_and_split_data(train_metadata, y_train)

# Convert training set into a TF dataset via generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
train_dataset = train_dataset.map(lambda x,y: load_image_data(x, y), 
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.cache()
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Convert validation set into a TF dataset via generator
validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
validation_dataset = validation_dataset.map(lambda x,y: load_image_data(x, y), 
                                            num_parallel_calls=tf.data.experimental.AUTOTUNE)

validation_dataset = validation_dataset.cache()
validation_dataset = validation_dataset.repeat()
validation_dataset = validation_dataset.batch(1)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Num. Total Images:  7409
Num. Train Images:  6669
Num. Validation Images:  740


In [13]:
def train_model(model, train_dataset, validation_dataset, name):    
    # Compile model                                                                                                      
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=4e-4),                                                           
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),                                  
                  metrics=['accuracy'])      
    
    # Stop early if we're not making good progress                                                                           
    early_stop_monitor = tf.keras.callbacks.EarlyStopping(monitor='val_loss',                                                                                              
                                                          restore_best_weights=True,                                                                                       
                                                          patience=10)   

    # Prepare for checkpoints            
    checkpoint_path = './checkpoints/' + name + '/cp-{epoch:04d}.ckpt'                                   
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,                                                                                    
                                                     verbose=1,                                                                                                   
                                                     save_weights_only=True,                                                                                     
                                                     save_freq=2500000)

    # Tensorboard                                                                                                        
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")                                              
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    history = model.fit(x=train_dataset,
                        epochs=NUM_EPOCHS,                                                                                                  
                        steps_per_epoch=TRAIN_STEPS_PER_EPOCH,
                        callbacks=[tensorboard_callback, cp_callback, early_stop_monitor],
                        use_multiprocessing=True,
                        validation_steps=num_validation,
                        validation_data=validation_dataset,
                        shuffle=True)

    return history

In [13]:
# Train and save model
train_model(mobilenet, train_dataset, validation_dataset, 'mobilenet_imbalanced')

Train for 5000 steps, validate for 740 steps
Epoch 1/5
5000/5000 [==============================] - 116s 23ms/step - loss: 2.5203 - accuracy: 0.6537 - val_loss: 2.6127 - val_accuracy: 0.5595
Epoch 2/5
5000/5000 [==============================] - 108s 22ms/step - loss: 2.3699 - accuracy: 0.8009 - val_loss: 2.5979 - val_accuracy: 0.5743
Epoch 3/5
5000/5000 [==============================] - 108s 22ms/step - loss: 2.3371 - accuracy: 0.8320 - val_loss: 2.6098 - val_accuracy: 0.5635
Epoch 4/5
5000/5000 [==============================] - 107s 21ms/step - loss: 2.3140 - accuracy: 0.8544 - val_loss: 2.5998 - val_accuracy: 0.5797
Epoch 5/5
5000/5000 [==============================] - 108s 22ms/step - loss: 2.2675 - accuracy: 0.9011 - val_loss: 2.5862 - val_accuracy: 0.5824


In [14]:
if not os.path.exists(os.path.join('models', 'mobilenet_imbalanced')):
    os.makedirs(os.path.join('models', 'mobilenet_imbalanced'))
    
mobilenet.save(os.path.join('models', 'mobilenet_imbalanced'))
print('model saved')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/mobilenet_imbalanced/assets
model saved


## (2) MOBILENET + STANDARD AUGMENTATIONS

In [15]:
mobilenet_std_aug = get_mobilenet()

/home/dennis/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [16]:
# TODO: We have to somehow incorporate the below with tf.Datasets
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range=45,
#     width_shift_range=0.4,
#     height_shift_range=0.4,
#     zoom_range=[0.4, 1.6],
#     horizontal_flip=True,
#     brightness_range=(0.6, 1.4),
#     fill_mode='nearest',
# )

# NOTE: Apply a map function to perform transformations rather than using ImageDataGen
@tf.function
def std_augment_image(img_tensor, label):
    transformed = img_tensor
    # Random rotation
#     if tf.random.uniform([]) <= 0.8:
#         angle = tf.random.uniform([]) * 45.0
#         transformed = tfa.image.rotate(transformed, angle)
    # Random zoom (5%)
    if tf.random.uniform([]) <= 0.05:
        crop_size = tf.random.uniform([], minval=0.4, maxval=0.8) * 64.0
        transformed = tf.image.resize(tf.image.random_crop(transformed, [crop_size, crop_size, 3]), [64, 64])
    # Random brightness adjustment
    if tf.random.uniform([]) <= 0.5:
        transformed = tf.image.random_brightness(transformed, 0.6)
    # Random horizontal flip
    transformed = tf.image.random_flip_up_down(transformed)
    return (transformed, label)

In [17]:
# Redfine train dataset
# Convert training set into a TF dataset via generator
train_dataset_std_aug = tf.data.Dataset.from_generator(
    lambda: train_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
train_dataset_std_aug = train_dataset_std_aug.map(lambda x,y: load_image_data(x, y), 
                                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset_std_aug = train_dataset_std_aug.cache()
train_dataset_std_aug = train_dataset_std_aug.map(std_augment_image,
                                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset_std_aug = train_dataset_std_aug.repeat()
train_dataset_std_aug = train_dataset_std_aug.batch(BATCH_SIZE)
train_dataset_std_aug = train_dataset_std_aug.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [18]:
# Train and save model
train_model(mobilenet_std_aug, train_dataset_std_aug, validation_dataset, 'mobilenet_imbalanced_std_aug')

Train for 5000 steps, validate for 740 steps
Epoch 1/5
5000/5000 [==============================] - 111s 22ms/step - loss: 2.6718 - accuracy: 0.4976 - val_loss: 2.6551 - val_accuracy: 0.5203
Epoch 2/5
5000/5000 [==============================] - 110s 22ms/step - loss: 2.5661 - accuracy: 0.6019 - val_loss: 2.6081 - val_accuracy: 0.5608
Epoch 3/5
5000/5000 [==============================] - 109s 22ms/step - loss: 2.5086 - accuracy: 0.6594 - val_loss: 2.5955 - val_accuracy: 0.5811
Epoch 4/5
5000/5000 [==============================] - 108s 22ms/step - loss: 2.4528 - accuracy: 0.7155 - val_loss: 2.5749 - val_accuracy: 0.5973
Epoch 5/5
5000/5000 [==============================] - 108s 22ms/step - loss: 2.4289 - accuracy: 0.7388 - val_loss: 2.5678 - val_accuracy: 0.6027


In [19]:
if not os.path.exists(os.path.join('models', 'mobilenet_imbalanced_std_aug')):
    os.makedirs(os.path.join('models', 'mobilenet_imbalanced_std_aug'))
    
mobilenet_std_aug.save(os.path.join('models', 'mobilenet_imbalanced_std_aug'))
print('model saved')

INFO:tensorflow:Assets written to: models/mobilenet_imbalanced_std_aug/assets
model saved


## (3) MOBILENET + BAGAN

In [8]:
# New model for bagan augmentation
mobilenet_bagan_aug = get_mobilenet()

/home/dennis/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [9]:
bagan_train_metadata = load_metadata('./metadata_output/bagan_train_metadata.txt')
combined_bagan_train_metadata = []
combined_bagan_train_metadata.extend(train_metadata)
combined_bagan_train_metadata.extend(bagan_train_metadata)

y_train_bagan = load_labels_with_encoder(combined_bagan_train_metadata, encoder)
(bagan_train_paths_and_labels, bagan_validation_paths_and_labels) = shuffle_and_split_data(combined_bagan_train_metadata, y_train_bagan)

Num. Total Images:  7709
Num. Train Images:  6939
Num. Validation Images:  770


In [14]:
# Redfine train dataset to include BAGAN samples
# Convert training set into a TF dataset via generator
train_dataset_bagan_aug = tf.data.Dataset.from_generator(
    lambda: bagan_train_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_bagan[0])]))
)
train_dataset_bagan_aug = train_dataset_bagan_aug.map(lambda x,y: load_image_data(x, y), 
                                                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset_bagan_aug = train_dataset_bagan_aug.cache()
train_dataset_bagan_aug = train_dataset_bagan_aug.repeat()
train_dataset_bagan_aug = train_dataset_bagan_aug.batch(BATCH_SIZE)
train_dataset_bagan_aug = train_dataset_bagan_aug.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Convert validation set into a TF dataset via generator
bagan_validation_dataset = tf.data.Dataset.from_generator(
    lambda: bagan_validation_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_bagan[0])]))
)
bagan_validation_dataset = bagan_validation_dataset.map(lambda x,y: load_image_data(x, y), 
                                                        num_parallel_calls=tf.data.experimental.AUTOTUNE)

bagan_validation_dataset = bagan_validation_dataset.cache()
bagan_validation_dataset = bagan_validation_dataset.repeat()
bagan_validation_dataset = bagan_validation_dataset.batch(1)
bagan_validation_dataset = bagan_validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [15]:
# Train and save model
train_model(mobilenet_bagan_aug, train_dataset_bagan_aug, bagan_validation_dataset, 'mobilenet_imbalanced_bagan_aug')

Train for 5000 steps, validate for 770 steps
Epoch 1/5
5000/5000 [==============================] - 117s 23ms/step - loss: 2.5430 - accuracy: 0.6309 - val_loss: 2.6112 - val_accuracy: 0.5714
Epoch 2/5
5000/5000 [==============================] - 114s 23ms/step - loss: 2.3731 - accuracy: 0.7979 - val_loss: 2.5815 - val_accuracy: 0.5896
Epoch 3/5
5000/5000 [==============================] - 114s 23ms/step - loss: 2.3099 - accuracy: 0.8596 - val_loss: 2.5804 - val_accuracy: 0.5870
Epoch 4/5
5000/5000 [==============================] - 115s 23ms/step - loss: 2.2912 - accuracy: 0.8773 - val_loss: 2.5674 - val_accuracy: 0.6000
Epoch 5/5
5000/5000 [==============================] - 115s 23ms/step - loss: 2.2803 - accuracy: 0.8877 - val_loss: 2.5595 - val_accuracy: 0.6052


In [16]:
if not os.path.exists(os.path.join('models', 'mobilenet_imbalanced_bagan_aug')):
    os.makedirs(os.path.join('models', 'mobilenet_imbalanced_bagan_aug'))
    
mobilenet_bagan_aug.save(os.path.join('models', 'mobilenet_imbalanced_bagan_aug'))
print('model saved')

INFO:tensorflow:Assets written to: models/mobilenet_imbalanced_bagan_aug/assets
model saved


## (4) MOBILENET + SINGAN

In [25]:
# New model for singan augmentation
mobilenet_singan_aug = get_mobilenet()

/home/dennis/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [26]:
singan_train_metadata = load_metadata('./metadata_output/singan_train_metadata.txt')
combined_train_metadata = []
combined_train_metadata.extend(train_metadata)
combined_train_metadata.extend(singan_train_metadata)

y_train_singan = load_labels_with_encoder(combined_train_metadata, encoder)
(singan_train_paths_and_labels, singan_validation_paths_and_labels) = shuffle_and_split_data(combined_train_metadata, y_train_singan)

Num. Total Images:  7909
Num. Train Images:  7119
Num. Validation Images:  790


In [27]:
# Redfine train dataset to include SinGAN samples
# Convert training set into a TF dataset via generator
train_dataset_singan_aug = tf.data.Dataset.from_generator(
    lambda: singan_train_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_singan[0])]))
)
train_dataset_singan_aug = train_dataset_singan_aug.map(lambda x,y: load_image_data(x, y), 
                                                        num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset_singan_aug = train_dataset_singan_aug.cache()
train_dataset_singan_aug = train_dataset_singan_aug.repeat()
train_dataset_singan_aug = train_dataset_singan_aug.batch(BATCH_SIZE)
train_dataset_singan_aug = train_dataset_singan_aug.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Convert validation set into a TF dataset via generator
singan_validation_dataset = tf.data.Dataset.from_generator(
    lambda: singan_validation_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_singan[0])]))
)
singan_validation_dataset = singan_validation_dataset.map(lambda x,y: load_image_data(x, y), 
                                                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

singan_validation_dataset = singan_validation_dataset.cache()
singan_validation_dataset = singan_validation_dataset.repeat()
singan_validation_dataset = singan_validation_dataset.batch(1)
singan_validation_dataset = singan_validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [28]:
# Train and save model
train_model(mobilenet_singan_aug, train_dataset_singan_aug, singan_validation_dataset, 'mobilenet_imbalanced_singan_aug')

Train for 5000 steps, validate for 790 steps
Epoch 1/5
5000/5000 [==============================] - 116s 23ms/step - loss: 2.5125 - accuracy: 0.6602 - val_loss: 2.5630 - val_accuracy: 0.6076
Epoch 2/5
5000/5000 [==============================] - 113s 23ms/step - loss: 2.3342 - accuracy: 0.8373 - val_loss: 2.5244 - val_accuracy: 0.6544
Epoch 3/5
5000/5000 [==============================] - 114s 23ms/step - loss: 2.2705 - accuracy: 0.8999 - val_loss: 2.5251 - val_accuracy: 0.6494
Epoch 4/5
5000/5000 [==============================] - 114s 23ms/step - loss: 2.2500 - accuracy: 0.9192 - val_loss: 2.5248 - val_accuracy: 0.6481
Epoch 5/5
5000/5000 [==============================] - 112s 22ms/step - loss: 2.2392 - accuracy: 0.9291 - val_loss: 2.5241 - val_accuracy: 0.6506


In [29]:
if not os.path.exists(os.path.join('models', 'mobilenet_imbalanced_singan_aug')):
    os.makedirs(os.path.join('models', 'mobilenet_imbalanced_singan_aug'))
    
mobilenet_singan_aug.save(os.path.join('models', 'mobilenet_imbalanced_singan_aug'))
print('model saved')

INFO:tensorflow:Assets written to: models/mobilenet_imbalanced_singan_aug/assets
model saved


# Model Evaluation

In this section, we will evaluate the models against the holdout data to compare performance.

## Load Models

In [30]:
mobilenet_singan_eval = tf.keras.models.load_model('models/mobilenet_imbalanced_singan_aug')

In [29]:
mobilenet_imbalanced_eval = tf.keras.models.load_model('models/mobilenet_imbalanced')

In [17]:
mobilenet_bagan_eval = tf.keras.models.load_model('models/mobilenet_imbalanced_bagan_aug')

In [30]:
mobilenet_imbalanced_std_aug_eval = tf.keras.models.load_model('models/mobilenet_imbalanced_std_aug')

### Load Test Data

In [18]:
holdout_metadata = load_metadata('./metadata_output/filtered_test_metadata.txt')
y_train_holdout = load_labels_with_encoder(holdout_metadata, encoder)
holdout_paths_and_labels = join_paths_and_labels(holdout_metadata, y_train_holdout)
holdout_size = len(holdout_paths_and_labels)

In [19]:
# Holdout dataset
holdout_dataset = tf.data.Dataset.from_generator(
    lambda: holdout_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_holdout[0])]))
)
holdout_dataset = holdout_dataset.map(lambda x,y: load_image_data(x, y), 
                                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

holdout_dataset = holdout_dataset.cache()
holdout_dataset = holdout_dataset.repeat()
holdout_dataset = holdout_dataset.batch(BATCH_SIZE)
holdout_dataset = holdout_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Evaluate Models

In [20]:
def evaluate_against_holdout(model, dataset=holdout_dataset, steps=holdout_size):
    return model.evaluate(dataset, steps=steps, use_multiprocessing=True)

In [34]:
# Baseline
evaluate_against_holdout(mobilenet_imbalanced_eval)

3208/3208 [==============================] - 57s 18ms/step - loss: 2.5632 - accuracy: 0.6035


[2.5631853636363497, 0.60349125]

In [35]:
# Simple Augmentation
evaluate_against_holdout(mobilenet_imbalanced_std_aug_eval)

3208/3208 [==============================] - 56s 18ms/step - loss: 2.5659 - accuracy: 0.6060


[2.565883205418575, 0.60598505]

In [21]:
# BAGAN (Full)
evaluate_against_holdout(mobilenet_bagan_eval)

3208/3208 [==============================] - 58s 18ms/step - loss: 2.5417 - accuracy: 0.6241


[2.541655621921035, 0.62406486]

In [31]:
# SinGAN
evaluate_against_holdout(mobilenet_singan_eval)

3208/3208 [==============================] - 58s 18ms/step - loss: 2.5312 - accuracy: 0.6372


[2.5312455402050826, 0.6371571]

In [22]:
# Filter only to interested categories
filtered_holdout_paths_and_labels = [x for x in holdout_paths_and_labels if np.any(np.take(x[1], interested_indices))]
num_filtered_holdout = len(filtered_holdout_paths_and_labels)
print("Total filtered samples: {}".format(num_filtered_holdout))

Total filtered samples: 283


In [23]:
# Filtered Holdout dataset
holdout_interested_dataset = tf.data.Dataset.from_generator(
    lambda: filtered_holdout_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train_holdout[0])]))
)
holdout_interested_dataset = holdout_interested_dataset.map(lambda x,y: load_image_data(x, y), 
                                                            num_parallel_calls=tf.data.experimental.AUTOTUNE)

holdout_interested_dataset = holdout_interested_dataset.cache()
holdout_interested_dataset = holdout_interested_dataset.repeat()
holdout_interested_dataset = holdout_interested_dataset.batch(BATCH_SIZE)
holdout_interested_dataset = holdout_interested_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [39]:
# Baseline
evaluate_against_holdout(mobilenet_imbalanced_eval, holdout_interested_dataset, num_filtered_holdout)

283/283 [==============================] - 5s 18ms/step - loss: 2.7871 - accuracy: 0.3746


[2.7871253692640434, 0.3745583]

In [40]:
# Simple Augmentation
evaluate_against_holdout(mobilenet_imbalanced_std_aug_eval, holdout_interested_dataset, num_filtered_holdout)

283/283 [==============================] - 5s 17ms/step - loss: 3.1604 - accuracy: 0.0000e+00


[3.160352815587613, 0.0]

In [24]:
# BAGAN (Full)
evaluate_against_holdout(mobilenet_bagan_eval, holdout_interested_dataset, num_filtered_holdout)

283/283 [==============================] - 5s 18ms/step - loss: 2.4836 - accuracy: 0.6820


[2.483620049675446, 0.6819788]

In [32]:
# SinGAN
evaluate_against_holdout(mobilenet_singan_eval, holdout_interested_dataset, num_filtered_holdout)

283/283 [==============================] - 5s 18ms/step - loss: 2.3952 - accuracy: 0.7739


[2.39515260672822, 0.7738516]